In [1]:
import torch
from det3d.torchie import Config

import pickle

config = "/workspace/centerformer/configs/nusc/nuscenes_centerformer_poolformer.py"

cfg = Config.fromfile(config)
FINDCENTER_GEN_ONNX = True
POOLFORMER_GEN_ONNX = False

In [2]:
from det3d.models import build_detector
from det3d.torchie.trainer import load_checkpoint

checkpoint_path = "/workspace/centerformer/work_dirs/nuscenes_poolformer/poolformer.pth"

model = build_detector(cfg.model, train_cfg=None, test_cfg=cfg.test_cfg)
checkpoint = load_checkpoint(model, checkpoint_path, map_location="cpu");

/data/centerformer/det3d/core/bbox/geometry.py:149: NumbaDeprecationWarning: The keyword argument 'nopython=False' was supplied. From Numba 0.59.0 the default is being changed to True and use of 'nopython=False' will raise a warning as the argument will have no effect. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit(nopython=False)
/data/centerformer/det3d/core/bbox/geometry.py:162: NumbaDeprecationWarning: The keyword argument 'nopython=False' was supplied. From Numba 0.59.0 the default is being changed to True and use of 'nopython=False' will raise a warning as the argument will have no effect. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit(nopython=False)
/data/centerformer/det3d/core/bbox/geometry.py:280: NumbaDeprecationWarning: The 'nopython' keyword argument was

Deformable Convolution not built!
Deformable Convolution not built!
[12/14/2023-07:08:44] [TRT] [W] CUDA lazy loading is not enabled. Enabling it can significantly reduce device memory usage. See `CUDA_MODULE_LOADING` in https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#env-vars
Use HM Bias:  -2.19


In [3]:
from torch import nn

class CenterFinder(nn.Module):
    def __init__(self, model):
        super(CenterFinder, self).__init__()
        self.model = model

    def forward(self, x):
        # x = self.model.neck(x)
        # preds = self.model.bbox_head(x)
        # return preds
        return self.model.neck.find_centers(x) # pos_features, out_dict_list, ct_feat 
    
centerFinder = CenterFinder(model)
# centerFinder.cuda()
centerFinder.eval();

In [4]:
import onnx
import numpy as np

if FINDCENTER_GEN_ONNX:
    model_name = "findCenter.onnx"
    # dummy_input=torch.randn(x.shape, requires_grad=True).cuda()
    dummy_input = torch.as_tensor(np.ones([1, 256, 180, 180]), dtype=torch.float32)
    torch.onnx.export(centerFinder, (dummy_input), model_name,
                input_names=['input_tensor'], 
                output_names=['ct_feat', 'center_pos_embedding', 'out_scores', 'out_labels', 'out_orders', 'out_masks'],
                export_params=True, 
                do_constant_folding=True,
                opset_version=14,
                )
    
    # !export POLYGRAPHY_AUTOINSTALL_DEPS=1
    !polygraphy surgeon sanitize findCenter.onnx --fold-constants --output findCenter_folded.onnx

    onnx.checker.check_model(onnx.load("findCenter_folded.onnx"))
    print("gen findCenter.onnx success!")
else:
    print("pass")

/data/centerformer/det3d/models/necks/rpn_transformer_multitask.py:940: TracerWarning: Converting a tensor to a NumPy array might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  scores = scores.detach().cpu().numpy()
/data/centerformer/det3d/models/necks/rpn_transformer_multitask.py:942: TracerWarning: torch.from_numpy results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  order = torch.from_numpy(order).to(labels.device)
/data/centerformer/det3d/models/necks/rpn_transformer_multitask.py:943: TracerWarning: torch.from_numpy results are registered as constants in the trace. You can safely ignore this warning if yo

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

[W] 'colored' module is not installed, will not use colors when logging. To enable colors, please install the 'colored' module: python3 -m pip install colored
[I] RUNNING | Command: /usr/local/bin/polygraphy surgeon sanitize findCenter.onnx --fold-constants --output findCenter_folded.onnx
[I] Inferring shapes in the model with `onnxruntime.tools.symbolic_shape_infer`.
    Note: To force Polygraphy to use `onnx.shape_inference` instead, set `allow_onnxruntime=False` or use the `--no-onnxruntime-shape-inference` command-line option.
[I] Loading model: /data/centerformer/work_dirs/partition/onnx/findCenter.onnx
[I] Original Model:
    Name: torch_jit | ONNX Opset: 14
    
    ---- 1 Graph Input(s) ----
    {input_tensor [dtype=float32, shape=(1, 256, 180, 180)]}
    
    ---- 6 Graph Output